# Day6_0: pandas 고급 조작 (groupby, pivot, merge) - 정답

## 📚 학습 목표

**Part 1: 기초**
1. groupby로 기본 집계 수행하기 (sum, mean, count)
2. 다중 컬럼 groupby로 세분화된 집계하기
3. agg() 함수로 복합 집계 실행하기
4. transform() vs apply() 차이 이해하기
5. Plotly로 집계 결과 시각화하기

**Part 2: 심화**
1. pivot_table()로 데이터 요약하기
2. merge()로 데이터 조인하기 (inner, outer, left, right)
3. concat()으로 데이터 결합하기
4. px.imshow()로 피벗 테이블 히트맵 그리기

---

## 🎯 실습 퀴즈 정답

---

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

### Q1. 기본 groupby 집계 ⭐

**문제**: 다음 매출 데이터에서 지역별 총 매출을 계산하세요.

In [ ]:
# Q1 정답

data = pd.DataFrame({
    'region': ['서울', '부산', '서울', '대구', '부산'],
    'sales': [100, 150, 120, 90, 180]
})

# 지역별 총 매출 계산
result = data.groupby('region')['sales'].sum()

print("지역별 총 매출:")
print(result)

In [ ]:
# 테스트
assert result['서울'] == 220, "서울 매출이 잘못되었습니다"
assert result['부산'] == 330, "부산 매출이 잘못되었습니다"
assert result['대구'] == 90, "대구 매출이 잘못되었습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `groupby('region')`으로 지역별로 그룹화
- `['sales']`로 매출 컬럼만 선택
- `sum()`으로 각 그룹의 합계 계산

**핵심 개념**:
- **groupby()**: Split-Apply-Combine 패턴의 기본
- 결과는 **Series** 형태 (인덱스가 지역명)

**대안 솔루션**:
```python
# DataFrame 형태로 받기
result = data.groupby('region')['sales'].sum().reset_index()
result.columns = ['지역', '총매출']
```

**흔한 실수**:
- ❌ `data.groupby('region').sum()`: 모든 숫자 컬럼이 집계됨
- ❌ `data['sales'].groupby('region')`: 순서가 바뀌면 오류

**실무 팁**:
- 시각화할 때는 `.reset_index()`로 DataFrame 변환 후 사용
- 여러 컬럼 집계 시 `[['sales', 'quantity']].sum()` 형태로 사용

---

### Q2. 다중 컬럼 groupby ⭐⭐

**문제**: 지역과 제품별 총 매출을 계산하고 DataFrame으로 변환하세요.

In [ ]:
# Q2 정답

data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'product': ['노트북', '마우스', '노트북', '마우스'],
    'sales': [1200, 30, 1000, 25]
})

# 지역 + 제품별 총 매출
result = data.groupby(['region', 'product'])['sales'].sum().reset_index()

print("지역 + 제품별 총 매출:")
print(result)

In [ ]:
# 테스트
assert len(result) == 4, "행 개수가 잘못되었습니다"
assert list(result.columns) == ['region', 'product', 'sales'], "컬럼명이 잘못되었습니다"
seoul_laptop = result[(result['region'] == '서울') & (result['product'] == '노트북')]['sales'].values[0]
assert seoul_laptop == 1200, "서울 노트북 매출이 잘못되었습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `groupby(['region', 'product'])`로 다중 컬럼 그룹화
- `reset_index()`로 Series를 DataFrame으로 변환

**핵심 개념**:
- **다중 컬럼 groupby**: 리스트로 여러 컬럼 지정
- **reset_index()**: 인덱스를 일반 컬럼으로 변환

**대안 솔루션**:
```python
# as_index=False로 바로 DataFrame 형태
result = data.groupby(['region', 'product'], as_index=False)['sales'].sum()
```

**흔한 실수**:
- ❌ `reset_index()` 없이 사용: MultiIndex Series가 되어 불편
- ❌ 컬럼 순서 바뀜: groupby 리스트 순서가 중요

**실무 팁**:
- 세분화된 분석에 필수 ("서울의 노트북 매출은?")
- 3개 이상 그룹화도 가능: `['region', 'product', 'channel']`

---

### Q3. agg() 복합 집계 ⭐⭐

**문제**: 제품별로 매출의 합계, 평균, 최대값을 동시에 계산하세요.

In [ ]:
# Q3 정답

data = pd.DataFrame({
    'product': ['노트북', '노트북', '마우스', '마우스'],
    'sales': [1200, 1500, 30, 25]
})

# 제품별 복합 집계
result = data.groupby('product')['sales'].agg(['sum', 'mean', 'max'])

print("제품별 매출 통계:")
print(result)

In [ ]:
# 테스트
assert result.loc['노트북', 'sum'] == 2700, "노트북 합계가 잘못되었습니다"
assert result.loc['노트북', 'mean'] == 1350, "노트북 평균이 잘못되었습니다"
assert result.loc['마우스', 'max'] == 30, "마우스 최대값이 잘못되었습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `agg()`에 함수 이름 리스트 전달
- 결과는 컬럼이 함수명인 DataFrame

**핵심 개념**:
- **agg()**: 여러 집계 함수를 동시에 적용
- 리스트로 함수명 전달: `['sum', 'mean', 'max']`

**대안 솔루션**:
```python
# 컬럼명 한글로 변경
result = data.groupby('product')['sales'].agg(['sum', 'mean', 'max'])
result.columns = ['합계', '평균', '최대']

# 사용자 정의 함수도 가능
result = data.groupby('product')['sales'].agg([np.sum, np.mean, lambda x: x.max()])
```

**흔한 실수**:
- ❌ `agg('sum', 'mean')`: 괄호 없이 쓰면 오류
- ❌ 문자열과 함수 섞어 쓰기: 일관성 유지 필요

**실무 팁**:
- 월말 리포트에서 합계+평균+최대 한 번에 계산
- `.round(2)`로 소수점 정리하면 보기 좋음

---

### Q4. transform()으로 그룹 평균 추가 ⭐⭐⭐

**문제**: 지역별 평균 매출을 각 행에 추가한 후, 평균 대비 비율(%)을 계산하세요.

In [ ]:
# Q4 정답

data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산'],
    'sales': [100, 120, 150, 180]
})

# 1. 지역별 평균을 각 행에 추가
data['region_avg'] = data.groupby('region')['sales'].transform('mean')

# 2. 평균 대비 비율 계산
data['ratio'] = (data['sales'] / data['region_avg'] * 100).round(1)

print("결과:")
print(data)

In [ ]:
# 테스트
assert 'region_avg' in data.columns, "region_avg 컬럼이 없습니다"
assert 'ratio' in data.columns, "ratio 컬럼이 없습니다"
assert data.loc[0, 'region_avg'] == 110, "서울 평균이 잘못되었습니다"
assert abs(data.loc[0, 'ratio'] - 90.9) < 0.1, "비율 계산이 잘못되었습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `transform('mean')`으로 그룹별 평균을 원본 크기로 반환
- 평균과 원본값을 나누어 비율 계산

**핵심 개념**:
- **transform()**: 집계 결과를 원본과 같은 크기로 반환
- 각 행이 속한 그룹의 통계값을 행별로 할당

**대안 솔루션**:
```python
# apply + lambda 사용
data['region_avg'] = data.groupby('region')['sales'].transform(lambda x: x.mean())

# merge 방식 (느림)
avg_df = data.groupby('region')['sales'].mean().reset_index()
avg_df.columns = ['region', 'region_avg']
data = pd.merge(data, avg_df, on='region')
```

**흔한 실수**:
- ❌ `transform()` 없이 `groupby().mean()`: 크기가 줄어듦
- ❌ 비율 계산 시 0으로 나누기: 평균이 0인 경우 주의

**실무 팁**:
- 이상치 탐지에 유용: "평균보다 2배 이상 높은 매출 찾기"
- Z-score 계산에도 사용: `(x - mean) / std`

---

### Q5. pivot_table 생성 ⭐⭐⭐

**문제**: 지역(행) x 제품(열) 형태의 피벗 테이블을 만들고, 결측치를 0으로 채우세요.

In [ ]:
# Q5 정답

data = pd.DataFrame({
    'region': ['서울', '서울', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드'],
    'sales': [1200, 30, 1000, 80]
})

# 피벗 테이블 생성
result = data.pivot_table(
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0
)

print("피벗 테이블:")
print(result)

In [ ]:
# 테스트
assert result.loc['서울', '노트북'] == 1200, "서울 노트북이 잘못되었습니다"
assert result.loc['대구', '노트북'] == 0, "결측치가 0으로 채워지지 않았습니다"
assert result.loc['부산', '마우스'] == 0, "결측치가 0으로 채워지지 않았습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `pivot_table()`에 행(index), 열(columns), 값(values) 지정
- `fill_value=0`으로 결측치 채우기

**핵심 개념**:
- **pivot_table()**: 2차원 요약 테이블 생성
- **fill_value**: NaN을 특정 값으로 대체

**대안 솔루션**:
```python
# pivot() 사용 (집계 없이 재구조화만)
result = data.pivot(index='region', columns='product', values='sales').fillna(0)

# margins=True로 총합 추가
result = data.pivot_table(
    values='sales',
    index='region',
    columns='product',
    aggfunc='sum',
    fill_value=0,
    margins=True
)
```

**흔한 실수**:
- ❌ `fill_value` 없이 사용: NaN이 계산에 오류 유발
- ❌ `pivot()`과 `pivot_table()` 혼동: pivot은 중복 불가

**실무 팁**:
- 엑셀 피벗 테이블과 동일한 결과
- `aggfunc`를 여러 개 지정 가능: `['sum', 'mean']`

---

### Q6. Inner Merge ⭐⭐⭐

**문제**: 주문 데이터와 제품 데이터를 product_id 기준으로 Inner Join하세요.

In [ ]:
# Q6 정답

orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102, 104],
    'product_name': ['노트북', '마우스', '키보드'],
    'price': [1200, 30, 80]
})

# Inner Join
result = pd.merge(orders, products, on='product_id', how='inner')

print("Inner Join 결과:")
print(result)

In [ ]:
# 테스트
assert len(result) == 2, "Inner Join은 2개 행만 반환해야 합니다"
assert 103 not in result['product_id'].values, "product_id 103은 products에 없으므로 제외되어야 합니다"
assert 104 not in result['product_id'].values, "product_id 104는 orders에 없으므로 제외되어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `pd.merge()`로 두 DataFrame 결합
- `on='product_id'`로 조인 키 지정
- `how='inner'`로 교집합만 반환

**핵심 개념**:
- **Inner Join**: 양쪽 모두 존재하는 키만 포함
- product_id 103 (orders만), 104 (products만)는 제외됨

**대안 솔루션**:
```python
# how 생략 시 기본값이 inner
result = pd.merge(orders, products, on='product_id')

# 컬럼명이 다를 때
result = pd.merge(orders, products, left_on='product_id', right_on='prod_id')
```

**흔한 실수**:
- ❌ `how='left'`와 혼동: left는 왼쪽 모두 포함
- ❌ 조인 후 중복 확인 안 함: 1:N 관계일 때 행 증가

**실무 팁**:
- 양쪽 모두 있는 데이터만 필요할 때 사용
- 데이터 검증용으로 활용: "주문된 제품만 분석"

---

### Q7. concat으로 데이터 결합 ⭐⭐⭐⭐

**문제**: 1월과 2월 매출 데이터를 위아래로 결합하고, 인덱스를 재설정하세요.

In [ ]:
# Q7 정답

jan_sales = pd.DataFrame({
    'product': ['노트북', '마우스'],
    'sales': [1200, 30]
})

feb_sales = pd.DataFrame({
    'product': ['노트북', '키보드'],
    'sales': [1350, 85]
})

# 위아래 결합 + 인덱스 재설정
result = pd.concat([jan_sales, feb_sales], axis=0, ignore_index=True)

print("결합 결과:")
print(result)

In [ ]:
# 테스트
assert len(result) == 4, "4개 행이어야 합니다"
assert list(result.index) == [0, 1, 2, 3], "인덱스가 0, 1, 2, 3이어야 합니다"
assert result.loc[2, 'product'] == '노트북', "3번째 행은 2월 노트북이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `pd.concat()`에 DataFrame 리스트 전달
- `axis=0`으로 위아래 결합
- `ignore_index=True`로 인덱스 0부터 재설정

**핵심 개념**:
- **concat()**: 여러 DataFrame을 결합
- **axis=0**: 위아래 결합 (행 방향)
- **ignore_index**: 기존 인덱스 무시하고 새로 부여

**대안 솔루션**:
```python
# append() 사용 (deprecated)
result = jan_sales.append(feb_sales, ignore_index=True)

# keys로 월 구분
result = pd.concat([jan_sales, feb_sales], keys=['1월', '2월'])
```

**흔한 실수**:
- ❌ `ignore_index=True` 없이 사용: 인덱스 중복
- ❌ `axis=1`로 잘못 지정: 좌우 결합됨

**실무 팁**:
- 여러 파일(월별, 지역별)을 하나로 통합할 때 필수
- 리스트 컴프리헨션으로 여러 파일 한 번에: `pd.concat([pd.read_csv(f) for f in files])`

---

### Q8. Left Merge + 계산 ⭐⭐⭐⭐

**문제**: 주문 데이터에 제품 정보를 Left Join한 후, total_amount (수량 x 가격) 컬럼을 추가하세요.

In [ ]:
# Q8 정답

orders = pd.DataFrame({
    'order_id': [1, 2, 3],
    'product_id': [101, 102, 103],
    'quantity': [2, 5, 1]
})

products = pd.DataFrame({
    'product_id': [101, 102],
    'product_name': ['노트북', '마우스'],
    'price': [1200, 30]
})

# 1. Left Join
result = pd.merge(orders, products, on='product_id', how='left')

# 2. total_amount 계산
result['total_amount'] = result['quantity'] * result['price']

print("결과:")
print(result)

In [ ]:
# 테스트
assert len(result) == 3, "Left Join은 orders의 모든 행(3개)을 유지해야 합니다"
assert 'total_amount' in result.columns, "total_amount 컬럼이 없습니다"
assert result.loc[0, 'total_amount'] == 2400, "order_id 1의 금액이 잘못되었습니다"
assert pd.isna(result.loc[2, 'price']), "product_id 103은 매칭 안 되므로 NaN이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `how='left'`로 orders의 모든 행 유지
- products에 없는 product_id 103은 NaN으로 채워짐
- 계산 컬럼 추가

**핵심 개념**:
- **Left Join**: 왼쪽 DataFrame의 모든 행 보존
- 매칭 안 되는 값은 NaN

**대안 솔루션**:
```python
# fillna로 NaN 처리
result['total_amount'] = (result['quantity'] * result['price'].fillna(0))

# 한 번에 계산
result = pd.merge(orders, products, on='product_id', how='left')
result = result.assign(total_amount=lambda x: x['quantity'] * x['price'])
```

**흔한 실수**:
- ❌ NaN 처리 안 함: 계산 시 NaN 전파
- ❌ `how='inner'`와 혼동: 매칭 안 되는 주문 손실

**실무 팁**:
- VLOOKUP과 동일한 패턴 (왼쪽 기준)
- 주문 내역에 제품 정보 추가할 때 가장 많이 사용

---

### Q9. 다중 키 Merge ⭐⭐⭐⭐⭐

**문제**: 실제 매출과 목표 매출 데이터를 지역+제품 기준으로 조인하고, 달성률(%)을 계산하세요.

In [ ]:
# Q9 정답

actual = pd.DataFrame({
    'region': ['서울', '부산', '서울'],
    'product': ['노트북', '노트북', '마우스'],
    'sales': [1200, 1000, 30]
})

target = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'product': ['노트북', '노트북', '노트북'],
    'target': [1500, 900, 800]
})

# 1. 지역 + 제품 기준 Left Join
result = pd.merge(actual, target, on=['region', 'product'], how='left')

# 2. 달성률 계산
result['achievement_rate'] = (result['sales'] / result['target'] * 100).round(1)

print("결과:")
print(result)

In [ ]:
# 테스트
assert len(result) == 3, "actual의 모든 행(3개)을 유지해야 합니다"
assert 'achievement_rate' in result.columns, "achievement_rate 컬럼이 없습니다"
seoul_laptop_rate = result[(result['region'] == '서울') & (result['product'] == '노트북')]['achievement_rate'].values[0]
assert abs(seoul_laptop_rate - 80.0) < 0.1, "서울 노트북 달성률이 잘못되었습니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `on=['region', 'product']`로 다중 키 조인
- 실제/목표 비율로 달성률 계산

**핵심 개념**:
- **다중 키 조인**: 여러 컬럼을 동시에 매칭
- 리스트로 키 지정: `on=['key1', 'key2']`

**대안 솔루션**:
```python
# 목표가 없는 경우 0%로 처리
result['achievement_rate'] = (result['sales'] / result['target'].fillna(result['sales']) * 100).round(1)

# 색상 코딩 (100% 이상/미만)
result['status'] = result['achievement_rate'].apply(lambda x: '달성' if x >= 100 else '미달')
```

**흔한 실수**:
- ❌ 단일 키로 조인: 잘못된 매칭
- ❌ 목표가 0인 경우: ZeroDivisionError

**실무 팁**:
- 목표 달성률 분석의 핵심 패턴
- 지역+제품+월 등 3개 이상 키도 가능

---

### Q10. 종합 문제: groupby + pivot + merge ⭐⭐⭐⭐⭐

**문제**: 다음 거래 데이터에서:
1. 지역별 총 매출을 계산하고
2. 목표 데이터와 조인한 후
3. 달성률을 계산하여
4. 지역을 인덱스로 하는 요약 테이블을 만드세요.

In [ ]:
# Q10 정답

transactions = pd.DataFrame({
    'region': ['서울', '서울', '부산', '부산', '대구'],
    'product': ['노트북', '마우스', '노트북', '키보드', '마우스'],
    'sales': [1200, 30, 1000, 80, 25]
})

targets = pd.DataFrame({
    'region': ['서울', '부산', '대구'],
    'target': [1500, 1200, 100]
})

# 1. 지역별 총 매출 계산
regional_sales = transactions.groupby('region')['sales'].sum().reset_index()
regional_sales.columns = ['region', 'total_sales']

# 2. 목표 데이터와 조인
result = pd.merge(regional_sales, targets, on='region', how='left')

# 3. 달성률 계산
result['achievement_rate'] = (result['total_sales'] / result['target'] * 100).round(1)

# 4. 지역을 인덱스로 설정
result = result.set_index('region')

print("최종 결과:")
print(result)

In [ ]:
# 테스트
assert result.loc['서울', 'total_sales'] == 1230, "서울 총 매출이 잘못되었습니다"
assert result.loc['부산', 'total_sales'] == 1080, "부산 총 매출이 잘못되었습니다"
assert abs(result.loc['서울', 'achievement_rate'] - 82.0) < 0.1, "서울 달성률이 잘못되었습니다"
assert result.index.name == 'region', "인덱스 이름이 region이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. **groupby()**: 지역별 집계
2. **reset_index()**: DataFrame 변환
3. **merge()**: 목표 데이터와 조인
4. **계산**: 달성률 추가
5. **set_index()**: 지역을 인덱스로

**핵심 개념**:
- **데이터 파이프라인**: 여러 단계를 순차 실행
- 각 단계의 출력이 다음 단계의 입력

**대안 솔루션**:
```python
# 메소드 체이닝
result = (
    transactions
    .groupby('region')['sales'].sum()
    .reset_index()
    .rename(columns={'sales': 'total_sales'})
    .merge(targets, on='region', how='left')
    .assign(achievement_rate=lambda x: (x['total_sales'] / x['target'] * 100).round(1))
    .set_index('region')
)
```

**흔한 실수**:
- ❌ 중간에 변수 재할당 안 함: 원본 유지됨
- ❌ 컬럼명 충돌: `reset_index()` 후 컬럼명 확인

**실무 팁**:
- 전형적인 실무 분석 패턴: 집계 → 조인 → 계산 → 정리
- 메소드 체이닝으로 깔끔하게 작성 가능
- 각 단계마다 중간 결과 확인 권장

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|----------------|----------|
| groupby 기본 | `df.groupby('컬럼')['값'].sum()` | 지역별, 제품별 집계 |
| 다중 컬럼 groupby | `df.groupby(['컬럼1', '컬럼2'])` | 세분화된 분석 (지역+제품) |
| agg() 복합 집계 | `df.groupby().agg(['sum', 'mean'])` | 월말 리포트 (합계+평균+최대) |
| transform() | `df.groupby().transform('mean')` | 그룹 평균 vs 개별 비교 |
| Plotly 시각화 | `px.bar()`, `px.line()` | 집계 결과 차트 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| pivot_table | `df.pivot_table(values, index, columns)` | 엑셀 피벗 테이블처럼 요약 |
| merge (inner) | `pd.merge(df1, df2, on='key')` | 양쪽 모두 있는 데이터만 |
| merge (left) | `pd.merge(df1, df2, how='left')` | 왼쪽 기준 VLOOKUP |
| concat (axis=0) | `pd.concat([df1, df2])` | 여러 파일 위아래 결합 |
| concat (axis=1) | `pd.concat([df1, df2], axis=1)` | 좌우 결합 |
| px.imshow() | `px.imshow(pivot_table)` | 피벗 테이블 히트맵 |

### 💡 실무 팁

1. **groupby 후 reset_index()**: Series를 DataFrame으로 변환하여 시각화 쉽게
2. **pivot_table의 fill_value=0**: 결측치를 0으로 채워 계산 오류 방지
3. **merge의 how 선택**: inner(교집합), left(왼쪽 기준), outer(합집합) 명확히 구분
4. **concat의 ignore_index=True**: 인덱스 자동 재설정으로 깔끔하게
5. **transform vs apply**: 원본 크기 유지하려면 transform, 새 값 계산하려면 apply
6. **히트맵 컬러맵**: Blues(파랑), RdYlGn(빨강-노랑-초록), Viridis(보라-노랑) 상황에 맞게 선택
7. **엑셀 VLOOKUP 대신 merge**: 속도 빠르고 여러 키로 조인 가능